In [32]:
import os
import gc
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import tables

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import make_scorer

# Limit threads for numerical libraries to manage CPU usage
os.environ["OPENBLAS_NUM_THREADS"] = "7"
os.environ["OMP_NUM_THREADS"] = "7"

base_dir = "/home/skovtun/Python_projects/Kaggle/Single_cell/"
data_dir = os.path.join(base_dir, "data")
random_state = 77

os.chdir(data_dir)

In [12]:
#Getting external file providing the mapping of human genes to their chromosome coordinates on 
#the GRCh38 genome to use for reducing number of columns for every target.
# RAW LINE:
# 1	havana	gene	11869	14409	.	+	.	gene_id "ENSG00000223972"; gene_version "5"; gene_name "DDX11L1"; gene_source "havana"; gene_biotype "transcribed_unprocessed_pseudogene";

gtf_path = "Homo_sapiens.GRCh38.98.gtf"

genes = []

with open(gtf_path) as f:
    for line in f:
        #skipping comments
        if line.startswith("#"):
            continue
        #splitting the line
        fields = line.strip().split("\t")
        if fields[2] != "gene":
            continue
        
        chrom = fields[0]
        start = int(fields[3])
        end = int(fields[4])
        
        attr = fields[8]
        match_id = re.search(r'gene_id "([^"]+)"',attr)
        match_name = re.search(r'gene_name "([^"]+)"',attr)
        if match_id:
            gene_id = match_id.group(1)
        else:
            continue
        if match_name:
            gene_name = match_name.group(1)
        else:
            gene_name = None
        
        genes.append([gene_id, gene_name, chrom, start, end])

gene_df = pd.DataFrame(genes, columns=["gene_id", "gene_name", "chr", "start", "end"])
mapping = {str(i): f"chr{i}" for i in range(1,23)}
mapping['X'] = 'chrX'
mapping['Y'] = 'chrY'
gene_df['chr'] = gene_df['chr'].map(mapping).fillna(gene_df['chr'])
gene_df['chr'].unique()

array(['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chrX',
       'chr8', 'chr9', 'chr11', 'chr10', 'chr12', 'chr13', 'chr14',
       'chr15', 'chr16', 'chr17', 'chr18', 'chr20', 'chr19', 'chrY',
       'chr22', 'chr21', 'MT', 'KI270728.1', 'KI270727.1', 'KI270442.1',
       'GL000225.1', 'GL000009.2', 'GL000194.1', 'GL000205.2',
       'GL000195.1', 'KI270733.1', 'GL000219.1', 'GL000216.2',
       'KI270744.1', 'KI270734.1', 'GL000213.1', 'GL000220.1',
       'GL000218.1', 'KI270731.1', 'KI270750.1', 'KI270721.1',
       'KI270726.1', 'KI270711.1', 'KI270713.1'], dtype=object)

In [13]:
gene_df.shape

(60623, 5)

In [27]:
#Getting a list of genes and all coordinates, parsing ATAC Peaks
path = "train_multi_inputs.h5"
cols = pd.read_hdf(path, key="train_multi_inputs", start=0, stop=1).columns
path = "train_multi_targets.h5"
cols_t = pd.read_hdf(path, key = 'train_multi_targets', start = 0, stop=1).columns

#From the column names of the multi creating a dataframe with location name, start and end.
arr = pd.Series(cols.values)
r_p = r'([^:]+):([\d]+)-([\d]+)'
chr_ranges = arr.str.extract(r_p)
chr_ranges.columns = ['chr','start','end']
chr_ranges['start'] = chr_ranges['start'].astype(int)
chr_ranges['end'] = chr_ranges['end'].astype(int)

#Reducing gene_df by choosing onnly chr present in chr_ranges and only gene_id's present as targets.
multi_chr = list(chr_ranges['chr'].unique())
gene_df_multi = gene_df[gene_df['chr'].isin(multi_chr)]
gene_df_multi = gene_df_multi.set_index('gene_id')
missing = cols_t.difference(gene_df_multi.index)
gene_df_multi = gene_df_multi.loc[gene_df_multi.index.intersection(cols_t)]
gene_df_multi.shape

(23404, 4)

In [28]:
#calculating maximum amount of features for every target.
window = 200000

genes = gene_df_multi.loc[gene_df_multi.index.intersection(cols_t)].copy()
genes["left"]  = genes["start"] - window
genes["right"] = genes["end"]   + window
genes["gene_id"] = genes.index  # preserve gene_id as a column

chr_ranges = chr_ranges.rename(columns={
    "start": "start_peak",
    "end":   "end_peak"
})

target_cols = pd.DataFrame()
results = []
gene_id = []
start_peak = []
end_peak = []

for chr_name, chr_peaks in chr_ranges.groupby("chr"):
    sub_genes = genes[genes["chr"] == chr_name]
    if sub_genes.empty or chr_peaks.empty:
        continue

    merged = sub_genes[['gene_id', 'chr', 'left', 'right']].merge(chr_peaks[['chr', 'start_peak', 'end_peak']], on="chr")

    mask = (
        (merged["start_peak"] >= merged["left"]) &
        (merged["end_peak"]   <= merged["right"])
    )
    m = merged[mask]
    results.append(m)
        
target_cols = pd.concat(results)
target_cols['col'] = target_cols['chr'].astype(str)+":"+target_cols['start_peak'].astype(str)+"-"+target_cols['end_peak'].astype(str)

In [30]:
target_cols.shape

(1376144, 7)

In [29]:
target_cols.to_parquet("target_cols.parquet", index=False)

For every target we have from 1 to 234 features.

In [55]:
with tables.open_file("train_multi_inputs.h5", "r") as f:
    values = f.get_node("/train_multi_inputs/axis1")[:]
valid_ids = np.char.decode(values, encoding = 'utf-8')

metadata_old = pd.read_csv('metadata.csv', index_col = 'cell_id')
fix = pd.read_csv('metadata_cite_day_2_donor_27678.csv', index_col = 'cell_id')
metadata = pd.concat([metadata_old, fix], axis = 0)
del metadata_old, fix
multi_metadata  = metadata.loc[train_id,['day','donor']]
del metadata

In [53]:
# --- Data Splitting and Feature Separation ---
# Random Train-Test Split (80/20)
random_state=77
rng = np.random.default_rng(random_state)
shuffled_idx = rng.permutation(valid_ids)

split_point = int(len(shuffled_idx) * 0.8)
train_idx = shuffled_idx[:split_point]
test_idx = shuffled_idx[split_point:]


Writing a pipeline that will be computed for each of 23300 targets
First step: getting the data.
Genes and peaks correspondence are in target_cols
For now working on how to do everything for just one target.
First decision: reading all targets at once. array Y, with columns stored separately as items_Y.
Now getting peak values in batches: for 256 genes at once.

In [10]:
target_cols.iloc[728357]

chr                             chr19
gene_id               ENSG00000104848
start_peak                   49084891
end_peak                     49085777
col_t         chr19:49084891-49085777
col           chr19:49084891-49085777
Name: 8867055, dtype: object

In [13]:
gene_id = 'ENSG00000104848'
list_of_peak_strings = list(target_cols[target_cols['gene_id'] == gene_id]['col'])

In [5]:
import tables
import numpy as np
import time
start = time.time()
with tables.open_file("train_multi_targets.h5") as f:
    items_Y = f.root.train_multi_targets.block0_items[:].astype(str)  # gene_ids
    arr_Y   = f.root.train_multi_targets.block0_values                 # (n_cells, n_genes)
    Y = np.array(arr_Y, dtype=np.float32)                              # full target in RAM
print((time.time() - start)/60)

0.30817723274230957


In [22]:
items_Y[:5]

array(['ENSG00000121410', 'ENSG00000268895', 'ENSG00000175899',
       'ENSG00000245105', 'ENSG00000166535'], dtype='<U15')

In [21]:
Y[:5,:5]

array([[0.       , 0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       ],
       [0.       , 4.5079365, 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       ]],
      dtype=float32)

In [6]:
#Getting a list of genes and all coordinates
# path = "train_multi_inputs.h5"
# cols = pd.read_hdf(path, key="train_multi_inputs", start=0, stop=1).columns
# path = "train_multi_targets.h5"
# cols_t = pd.read_hdf(path, key = 'train_multi_targets', start = 0, stop=1).columns

peak_to_idx = {name: i for i, name in enumerate(cols)}
gene_to_idx = {name: i for i, name in enumerate(items_Y)}
gene_list = items_Y[:256]
list_of_peak_strings = list(target_cols[target_cols['gene_id'].isin(gene_list)]['col'])
selected_peaks_id = list(set([peak_to_idx[name] for name in list_of_peak_strings]))
print(selected_peaks_id[:5])


[39079, 50, 51, 32831, 64]


In [7]:
start_time = time.time()
col_idx =  selected_peaks_id    # your indices
batch = 2000

with tables.open_file("train_multi_inputs.h5", "r") as f:
    values = f.get_node("/train_multi_inputs/block0_values")
    items  = f.get_node("/train_multi_inputs/block0_items")[:]  # fully read

    n_rows = values.shape[0]
    n_cols = len(col_idx)

    arr = np.empty((n_rows, n_cols), dtype=values.dtype)

    for start in range(0, n_rows, batch):
        end = min(start + batch, n_rows)
        block = values[start:end, :]        # chunk-aligned read
        arr[start:end, :] = block[:, col_idx]

# now the file is closed; items is safe
df = pd.DataFrame(arr, columns=items[col_idx])
df.columns = df.columns.str.decode("utf-8")
print((time.time() - start_time)/60)

1.2264537493387857


In [37]:
df.head()

,chr13:95491683-95492594,KI270721.1:2091-2978,KI270721.1:8381-9254,chr12:942761-943471,chr10:100001240-100002159,chr10:100006137-100006993,chr10:100009448-100010341,chr10:100013840-100014756,chr10:100043298-100044049,chr1:33111718-33112623,...,chr7:20906021-20906882,chr7:20910868-20911751,chr7:20913372-20914288,chr13:95463279-95464129,chr7:20926434-20927110,chr7:20927658-20928561,chr13:95474906-95475770,chr13:95476873-95477605,chr13:95478266-95479176,chr13:95479928-95480825
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,5.213161,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,5.118048,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.544025,0.0,0.0,4.103673,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import pandas as pd

corr_max = []
y_train_var = []
y_test_var = []
corr_c = []

for i in range(256):
    gene_id = items_Y[i]

    # pick peaks for this gene
    peaks_str = list(target_cols.loc[target_cols['gene_id'] == gene_id, 'col'])

    # skip if no peaks
    if len(peaks_str) == 0:
        corr_max.append(np.nan)
        y_train_var.append(np.nan)
        y_test_var.append(np.nan)
        corr_c.append(np.nan)
        continue

    df_gene = df[peaks_str]

    # get expression for this gene
    y_gene = Y[:, i]

    # 1) max absolute peak–gene correlation
    y_series = pd.Series(y_gene, index=df_gene.index)
    corrs = df_gene.corrwith(y_series)
    corr_max.append(corrs.abs().max())

    # 2) train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        df_gene, y_gene, train_size=0.8, random_state=77
    )

    # 3) scale
    scaler = StandardScaler(with_mean=False)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # store variances
    y_train_var.append(np.var(y_train))
    y_test_var.append(np.var(y_test))

    # 4) ridge + test correlation
    model = Ridge(alpha=1.0, fit_intercept=True)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    corr_c.append(np.corrcoef(y_test, y_pred)[0, 1])
summary = pd.DataFrame({
    "gene_id": items_Y[:256],
    "max_peak_corr": corr_max,
    "y_train_var": y_train_var,
    "y_test_var": y_test_var,
    "ridge_test_r": corr_c,
})


In [9]:
import gc
gc.collect()

0

In [10]:
good = summary[
    (summary["max_peak_corr"].abs() > 0.05) &
    (summary["ridge_test_r"] > 0.1) &
    (summary["y_train_var"] > 0.5)
]
print(good)

             gene_id  max_peak_corr  y_train_var  y_test_var  ridge_test_r
13   ENSG00000115977       0.071275     6.815775    6.795776      0.139281
34   ENSG00000107331       0.056327     4.622707    4.605511      0.114302
48   ENSG00000115657       0.076905     1.428170    1.467709      0.148674
56   ENSG00000108846       0.084350     0.644879    0.687800      0.137206
57   ENSG00000125257       0.112532     8.042895    8.034110      0.283323
87   ENSG00000136379       0.061913     2.587219    2.633019      0.102538
102  ENSG00000099204       0.073839     6.154472    6.195627      0.136348
105  ENSG00000175164       0.050723     5.164624    5.191978      0.102156
113  ENSG00000166016       0.056345     2.810173    2.714567      0.138496
224  ENSG00000225792       0.065270     2.034776    1.949889      0.115125
240  ENSG00000265206       0.070217     6.259258    6.303818      0.129724


In [30]:
print(list(good['gene_id']))

['ENSG00000115977', 'ENSG00000107331', 'ENSG00000115657', 'ENSG00000108846', 'ENSG00000125257', 'ENSG00000136379', 'ENSG00000099204', 'ENSG00000175164', 'ENSG00000166016', 'ENSG00000225792', 'ENSG00000265206']


In [91]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import pandas as pd
from sklearn.linear_model import RidgeCV
from sklearn.metrics import make_scorer

corr_max = []
y_train_var = []
y_test_var = []
corr_c = []
corr_c_t = []

def pearsonr_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]

pearson_scorer = make_scorer(pearsonr_score, greater_is_better=True)

for gene_id in good_gene_id:

    # pick peaks for this gene
    peaks_str = list(target_cols.loc[target_cols['gene_id'] == gene_id, 'col'])

    # skip if no peaks
    if len(peaks_str) == 0:
        corr_max.append(np.nan)
        y_train_var.append(np.nan)
        y_test_var.append(np.nan)
        corr_c.append(np.nan)
        corr_c_t.append(np.nan)
        continue

    df_gene = df[peaks_str]
    
    # get expression for this gene
    y_gene = Y[:,good_gene_id[good_gene_id == gene_id].index[0]]

    # 1) max absolute peak–gene correlation
    y_series = pd.Series(y_gene, index=df_gene.index)
    corrs = df_gene.corrwith(y_series)
    corr_max.append(corrs.abs().max())

    # 2) train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        df_gene, y_gene, train_size=0.8, random_state=77
    )

    # 3) scale
    scaler = StandardScaler(with_mean=False)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # store variances
    y_train_var.append(np.var(y_train))
    y_test_var.append(np.var(y_test))

    # 4) ridge + test correlation
    model = Ridge(alpha=1.0, fit_intercept=True)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    corr_c.append(np.corrcoef(y_test, y_pred)[0, 1])

     # 5) ridge + tuning alpha
    alphas = np.logspace(-4, 4, 100)

    model = RidgeCV(alphas=alphas, scoring=pearson_scorer)
    model.fit(X_train_scaled, y_train)
    
    best_alpha = model.alpha_
    print("Best alpha:", best_alpha)
        
    model = Ridge(alpha=best_alpha, fit_intercept=True)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    corr_c_t.append(np.corrcoef(y_test, y_pred)[0, 1])

summary = pd.DataFrame({
    "gene_id": good_gene_id,
    "max_peak_corr": corr_max,
    "y_train_var": y_train_var,
    "y_test_var": y_test_var,
    "ridge_test_r": corr_c,
    "ridge_test_rt": corr_c_t,
})


Best alpha: 10000.0
Best alpha: 10000.0
Best alpha: 10000.0
Best alpha: 10000.0
Best alpha: 10000.0
Best alpha: 10000.0
Best alpha: 10000.0
Best alpha: 10000.0
Best alpha: 10000.0
Best alpha: 10000.0
Best alpha: 10000.0


In [92]:
print(summary)

             gene_id  max_peak_corr  y_train_var  y_test_var  ridge_test_r  \
13   ENSG00000115977       0.071275     6.815775    6.795776      0.139281   
34   ENSG00000107331       0.056327     4.622707    4.605511      0.114302   
48   ENSG00000115657       0.076905     1.428170    1.467709      0.148674   
56   ENSG00000108846       0.084350     0.644879    0.687800      0.137206   
57   ENSG00000125257       0.112532     8.042895    8.034110      0.283323   
87   ENSG00000136379       0.061913     2.587219    2.633019      0.102538   
102  ENSG00000099204       0.073839     6.154472    6.195627      0.136348   
105  ENSG00000175164       0.050723     5.164624    5.191978      0.102156   
113  ENSG00000166016       0.056345     2.810173    2.714567      0.138496   
224  ENSG00000225792       0.065270     2.034776    1.949889      0.115125   
240  ENSG00000265206       0.070217     6.259258    6.303818      0.129724   

     ridge_test_rt  
13        0.139407  
34        0.114481  


Tuning Ridge didn't help, let's try other models that could help with capturing non linearity.

In [17]:
good['gene_id'].index

Index([13, 34, 48, 56, 57, 87, 102, 105, 113, 224, 240], dtype='int64')

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import pandas as pd
from sklearn.metrics import make_scorer
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV

corr_max = []
y_train_var = []
y_test_var = []
corr_c = []
corr_c_t = []

def pearsonr_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]

pearson_scorer = make_scorer(pearsonr_score, greater_is_better=True)
good_gene_id = good['gene_id']
for gene_id in good_gene_id:

    # pick peaks for this gene
    peaks_str = list(target_cols.loc[target_cols['gene_id'] == gene_id, 'col'])

    # skip if no peaks
    if len(peaks_str) == 0:
        corr_max.append(np.nan)
        y_train_var.append(np.nan)
        y_test_var.append(np.nan)
        corr_c.append(np.nan)
        corr_c_t.append(np.nan)
        continue

    df_gene = df[peaks_str]
    
    # get expression for this gene
    y_gene = Y[:,good_gene_id[good_gene_id == gene_id].index[0]]

    # 1) max absolute peak–gene correlation
    y_series = pd.Series(y_gene, index=df_gene.index)
    corrs = df_gene.corrwith(y_series)
    corr_max.append(corrs.abs().max())

    # 2) train/test split, putting limitation on train size because of memory limitation
    X_train, X_test, y_train, y_test = train_test_split(
        df_gene, y_gene, train_size=10000, random_state=77
    )

    # 3) scale
    scaler = StandardScaler(with_mean=False)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # store variances
    y_train_var.append(np.var(y_train))
    y_test_var.append(np.var(y_test))

    # 4) ridge + test correlation
    model = Ridge(alpha=1.0, fit_intercept=True)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    corr_c.append(np.corrcoef(y_test, y_pred)[0, 1])

    # 5) Tuning KernelRidgee + test correlation
    param_grid = {
        "alpha": [0.1, 1, 10],
        "gamma": [1e-3, 1e-2],
        "kernel": ["rbf"],
    }
    
    kr = KernelRidge()
    
    grid = GridSearchCV(
        kr,
        param_grid,
        scoring=pearson_scorer,
        cv=3,
        n_jobs=-1
    )

    grid.fit(X_train_scaled, y_train)
    
    print("Best params:", grid.best_params_)
    best_model = grid.best_estimator_

    y_pred = best_model.predict(X_test_scaled)
    corr_c_t.append(np.corrcoef(y_test, y_pred)[0, 1])

summary = pd.DataFrame({
    "gene_id": good_gene_id,
    "max_peak_corr": corr_max,
    "y_train_var": y_train_var,
    "y_test_var": y_test_var,
    "ridge_test_r": corr_c,
    "kenel_ridge_test_rt": corr_c_t,
})


Best params: {'alpha': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Best params: {'alpha': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
Best params: {'alpha': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Best params: {'alpha': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Best params: {'alpha': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Best params: {'alpha': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Best params: {'alpha': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Best params: {'alpha': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Best params: {'alpha': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Best params: {'alpha': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Best params: {'alpha': 1, 'gamma': 0.001, 'kernel': 'rbf'}


In [13]:
print(summary)

             gene_id  max_peak_corr  y_train_var  y_test_var  ridge_test_r  \
13   ENSG00000115977       0.071275     6.765996    6.816464      0.122643   
34   ENSG00000107331       0.056327     4.655192    4.615505      0.085861   
48   ENSG00000115657       0.076905     1.487994    1.430646      0.111842   
56   ENSG00000108846       0.084350     0.648581    0.653983      0.106797   
57   ENSG00000125257       0.112532     7.976235    8.047922      0.277203   
87   ENSG00000136379       0.061913     2.511168    2.605215      0.079718   
102  ENSG00000099204       0.073839     6.094682    6.169738      0.115238   
105  ENSG00000175164       0.050723     5.195562    5.167503      0.084072   
113  ENSG00000166016       0.056345     2.841272    2.785895      0.110262   
224  ENSG00000225792       0.065270     2.106782    2.008544      0.097647   
240  ENSG00000265206       0.070217     6.246537    6.270519      0.114096   

     kenel_ridge_test_rt  
13              0.129831  
34       

Minimal gains, meaning most of the signal was captured by Ridge (linear)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import pandas as pd
from sklearn.metrics import make_scorer
import lightgbm as lgb



corr_max = []
y_train_var = []
y_test_var = []
corr_c = []
corr_c_t = []

def pearsonr_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]

pearson_scorer = make_scorer(pearsonr_score, greater_is_better=True)
good_gene_id = good['gene_id']
for gene_id in good_gene_id:

    # pick peaks for this gene
    peaks_str = list(target_cols.loc[target_cols['gene_id'] == gene_id, 'col'])

    # skip if no peaks
    if len(peaks_str) == 0:
        corr_max.append(np.nan)
        y_train_var.append(np.nan)
        y_test_var.append(np.nan)
        corr_c.append(np.nan)
        corr_c_t.append(np.nan)
        continue

    df_gene = df[peaks_str]
    
    # get expression for this gene
    y_gene = Y[:,good_gene_id[good_gene_id == gene_id].index[0]]

    # 1) max absolute peak–gene correlation
    y_series = pd.Series(y_gene, index=df_gene.index)
    corrs = df_gene.corrwith(y_series)
    corr_max.append(corrs.abs().max())

    # 2) train/test split, putting limitation on train size because of memory limitation
    X_train, X_test, y_train, y_test = train_test_split(
        df_gene, y_gene, train_size=0.8, random_state=77
    )

    # 3) scale
    scaler = StandardScaler(with_mean=False)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # store variances
    y_train_var.append(np.var(y_train))
    y_test_var.append(np.var(y_test))

    # 4) ridge + test correlation
    model = Ridge(alpha=1.0, fit_intercept=True)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    corr_c.append(np.corrcoef(y_test, y_pred)[0, 1])

    # 5) Tuning KernelRidgee + test correlation
    lgbm = lgb.LGBMRegressor(
        objective='regression',
        learning_rate=0.05,
        n_estimators=300,
        num_leaves=31,
        max_depth=6,
        min_data_in_leaf=50,
        feature_fraction=0.7,
        bagging_fraction=0.7,
        bagging_freq=1,
        n_jobs=-1,
        verbose=-1
    )
    lgbm.fit(X_train_scaled, y_train)
    y_pred = lgbm.predict(X_test_scaled)
    corr_c_t.append(np.corrcoef(y_test, y_pred)[0, 1])

summary = pd.DataFrame({
    "gene_id": good_gene_id,
    "max_peak_corr": corr_max,
    "y_train_var": y_train_var,
    "y_test_var": y_test_var,
    "ridge_test_r": corr_c,
    "lgbm_test_r": corr_c_t,
})


In [21]:
summary

,gene_id,max_peak_corr,y_train_var,y_test_var,ridge_test_r,lgbm_test_r
13,ENSG00000115977,0.071275,6.815775,6.795776,0.139281,0.166833
34,ENSG00000107331,0.056327,4.622707,4.605511,0.114302,0.135594
48,ENSG00000115657,0.076905,1.428170,1.467709,0.148674,0.146216
56,ENSG00000108846,0.084350,0.644879,0.687800,0.137206,0.117925
57,ENSG00000125257,0.112532,8.042895,8.034110,0.283323,0.291517
87,ENSG00000136379,0.061913,2.587219,2.633019,0.102538,0.103657
102,ENSG00000099204,0.073839,6.154472,6.195627,0.136348,0.136272
105,ENSG00000175164,0.050723,5.164624,5.191978,0.102156,0.107978
113,ENSG00000166016,0.056345,2.810173,2.714567,0.138496,0.166022
224,ENSG00000225792,0.065270,2.034776,1.949889,0.115125,0.105529


It 

In [63]:
#I am going to do train/test split only once and for each new batch just reuse indexes.
train_idx = X_train.index
test_idx = X_test.index

In [64]:
print(len(train_idx),len(test_idx),len(y_train),len(y_test))

84753 21189 84753 21189


X_train shape: (84753, 66)
X_test shape: (21189, 66)
y_train var: 0.06155252
y_test var: 0.06528162


-0.007252602796665639


In [ ]:
from sklearn.metrics import make_scorer
import numpy as np

def pearsonr_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]

pearson_scorer = make_scorer(pearsonr_score, greater_is_better=True)

Do not fogret to deal with gene_id's not present in external file.

In [18]:
print(missing)
print(len(missing))

Index(['ENSG00000198695', 'ENSG00000198712', 'ENSG00000198727',
       'ENSG00000198763', 'ENSG00000198786', 'ENSG00000198804',
       'ENSG00000198840', 'ENSG00000198886', 'ENSG00000198888',
       'ENSG00000198899', 'ENSG00000198938', 'ENSG00000212907',
       'ENSG00000228253', 'ENSG00000278704'],
      dtype='object', name='gene_id')
14
